In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
CLASSES = ['rock', 'paper', 'scissors']
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
def load_images_and_labels(image_path, target_size=(32,32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)
    label = tf.dtypes.cast(label, tf.float32)

    return image, label


In [4]:
def build_network():
    input_layer = Input(shape=(32,32,1))

    x = Conv2D(filters=32, kernel_size=(3,3), padding='same', strides=(1,1))(input_layer)
    x = ReLU()(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(units=3)(x)
    output = Softmax()(x)
    
    model = Model(inputs=input_layer, outputs=output)

    return model

In [5]:
def prepare_dataset(dataset_path,
                    buffer_size,
                    batch_size,
                    shuffle=True):
    dataset = (tf.data.Dataset
               .from_tensor_slices(dataset_path)
               .map(load_images_and_labels,
                    num_parallel_calls=AUTOTUNE))
    if shuffle:
        dataset.shuffle(buffer_size=buffer_size)

    dataset = (dataset.batch(batch_size=batch_size).prefetch(buffer_size=buffer_size))

    return dataset


In [6]:
files_pattern = ('C:/Users/hp/Documents/DATA/rockpaperscissors/*/*.png')
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

In [7]:
train_paths, test_paths = train_test_split(dataset_paths,
                                          test_size=0.2,
                                          random_state=999)
train_paths, val_paths = train_test_split(train_paths,
                                           test_size=0.2,
                                           random_state=999)

In [8]:
BATCH_SIZE = 1024
BUFFER_SIZE = 1024

train_dataset = prepare_dataset(train_paths,
                                buffer_size=BUFFER_SIZE,
                                batch_size=BATCH_SIZE)
validation_dataset = prepare_dataset(val_paths,
                                     buffer_size=BUFFER_SIZE,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True)
test_dataset = prepare_dataset(test_paths,
                               buffer_size=BUFFER_SIZE,
                               batch_size=BATCH_SIZE,
                               shuffle=True)


In [9]:
model = build_network()
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
EPOCHS = 250
model.fit(train_dataset, 
          validation_data=validation_dataset,
          epochs=EPOCHS)

Epoch 1/250


c:\Users\hp\anaconda3\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 4s 1s/step - loss: 1.3097 - accuracy: 0.3136 - val_loss: 1.1481 - val_accuracy: 0.3029
Epoch 2/250
2/2 [==============================] - 2s 812ms/step - loss: 1.1393 - accuracy: 0.3550 - val_loss: 1.3196 - val_accuracy: 0.3200
Epoch 3/250
2/2 [==============================] - 2s 783ms/step - loss: 1.3899 - accuracy: 0.3600 - val_loss: 1.2497 - val_accuracy: 0.3200
Epoch 4/250
2/2 [==============================] - 2s 783ms/step - loss: 1.2721 - accuracy: 0.3871 - val_loss: 1.0700 - val_accuracy: 0.5600
Epoch 5/250
2/2 [==============================] - 3s 870ms/step - loss: 1.0943 - accuracy: 0.3750 - val_loss: 1.1730 - val_accuracy: 0.3029
Epoch 6/250
2/2 [==============================] - 3s 852ms/step - loss: 1.1534 - accuracy: 0.3357 - val_loss: 1.1546 - val_accuracy: 0.3029
Epoch 7/250
2/2 [==============================] - 4s 1s/step - loss: 1.1090 - accuracy: 0.3364 - val_loss: 1.0504 - val_accuracy: 0.5029
Epoch 8/250
2/2 [==============

In [11]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

1/1 [==============================] - 1s 1s/step - loss: 0.1831 - accuracy: 0.9498
Loss: 0.18308620154857635, accuracy: 0.9497717022895813
